# Estimate Binomial draw probabilility

In [1]:
using DynamicHMCModels

Define a structure to hold the data.

In [2]:
struct BernoulliProblem
    "Total number of draws in the data."
    n::Int
    "Number of draws `==1` in the data"
    s::Vector{Int}
end;

Make the type callable with the parameters *as a single argument*.

In [3]:
function (problem::BernoulliProblem)((α, )::NamedTuple{(:α, )})
    @unpack n, s = problem        # extract the data
    loglikelihood(Binomial(n, α), s)
end

Create the data and complete setting up the problem.

In [4]:
obs = rand(Binomial(9, 2/3), 1)
p = BernoulliProblem(9, obs)
p((α = 0.5, ))

-2.6548056865833978

Write a function to return properly dimensioned transformation.

In [5]:
problem_transformation(p::BernoulliProblem) =
    as((α = as𝕀, ),  )
# Use a flat priors (the default, omitted) for α
P = TransformedLogDensity(problem_transformation(p), p)
∇P = LogDensityRejectErrors(ADgradient(:ForwardDiff, P));

Sample

In [6]:
chain, NUTS_tuned = NUTS_init_tune_mcmc(∇P, 1000)

MCMC, adapting ϵ (75 steps)
7.6e-6 s/step ...done
MCMC, adapting ϵ (25 steps)
8.7e-6 s/step ...done
MCMC, adapting ϵ (50 steps)
1.3e-5 s/step ...done
MCMC, adapting ϵ (100 steps)
0.00012 s/step ...done
MCMC, adapting ϵ (200 steps)
8.9e-6 s/step ...done
MCMC, adapting ϵ (400 steps)
8.9e-6 s/step ...done
MCMC, adapting ϵ (50 steps)
1.2e-5 s/step ...done
MCMC (1000 steps)
8.0e-6 s/step ...done


(NUTS_Transition{Array{Float64,1},Float64}[NUTS_Transition{Array{Float64,1},Float64}([0.1563411571569555], -6.513262432360197, 1, DynamicHMC.AdjacentTurn, 0.8422478223366457, 3), NUTS_Transition{Array{Float64,1},Float64}([0.19429491085529002], -3.851791414988219, 1, DynamicHMC.DoubledTurn, 1.0, 1), NUTS_Transition{Array{Float64,1},Float64}([0.3707861344162595], -3.588615028383155, 2, DynamicHMC.DoubledTurn, 1.0, 3), NUTS_Transition{Array{Float64,1},Float64}([-0.04182734045767772], -4.172119179185882, 1, DynamicHMC.DoubledTurn, 0.7837326161064258, 1), NUTS_Transition{Array{Float64,1},Float64}([0.43717741764688206], -3.8862619317216995, 2, DynamicHMC.DoubledTurn, 1.0, 3), NUTS_Transition{Array{Float64,1},Float64}([1.4082042233452254], -3.787750771671522, 1, DynamicHMC.AdjacentTurn, 0.90603200129354, 3), NUTS_Transition{Array{Float64,1},Float64}([-0.026013663068965187], -4.130728880680315, 2, DynamicHMC.DoubledTurn, 0.931662765870536, 3), NUTS_Transition{Array{Float64,1},Float64}([-0.8776

To get the posterior for ``α`` use `get_position` and then transform back.

In [7]:
posterior = TransformVariables.transform.(Ref(problem_transformation(p)), get_position.(chain));

Extract the parameter.

In [8]:
posterior_α = first.(posterior);

check the effective sample size

In [9]:
ess_α = effective_sample_size(posterior_α)

424.50444693702724

NUTS-specific statistics

In [10]:
NUTS_statistics(chain)

Hamiltonian Monte Carlo sample of length 1000
  acceptance rate mean: 0.93, min/25%/median/75%/max: 0.31 0.91 0.97 1.0 1.0
  termination: AdjacentTurn => 39% DoubledTurn => 61%
  depth: 1 => 59% 2 => 40% 3 => 1%


check the mean

In [11]:
mean(posterior_α)

0.7277474229158036

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*